Part #1 - Some Global Functions

Function A - Importing Market Data

In [ ]:
# n = Name of the excel file
# s = Sheet Name within the excel file
# r = Range of data which needs to be imported
# d = Data object that needs to be sent while exporting back

def import_excel(n,s,r):
    import xlwings as xw
    wb = xw.Book(n)
    sheet = wb.sheets[s]
    data = sheet.range(r).value
    return data 

def export_excel(n,s,r,d):
    import xlwings as xw
    wb = xw.Book(n)
    sheet = wb.sheets[s]
    sheet.range(r).value = d


In [2]:
data = import_excel('Market Data.xlsx','USDIRS','A3:B12')
#print(data)

Function B - MFBD Holiday Calendar

In [3]:
#dt = Date on which operation is to be performed
#cal1 = Country of Calendar 1, For USA pass argument US
#cal2 = Country of Calendar 2, For India pass argument IN .. This is an optional argument
# At present this function suports a max of 2 calendars only for MFBD

def mfbd(dt,cal1, cal2="NIL"):
    from datetime import date
    from datetime import timedelta
    import holidays
    calendar1 = holidays.country_holidays(cal1)
    if cal2 == "NIL":
        while dt.weekday() > 4 or dt in calendar1:
            dt = dt + timedelta(days=1)
    else:
        calendar2 = holidays.country_holidays(cal2)
        while dt.weekday() > 4 or dt in calendar1 or dt in calendar2:
            dt = dt + timedelta(days=1)
    #dt = date(2025,5,24)
    return dt


Function C - Curve Bootstrapping Function

Gives you a set of Discount Factors and Zero Rates for a given curve

In [ ]:
def bootstrap(data,i=-1,p=1):
    from datetime import date,timedelta
    import holidays
    import pandas as pd
    #PricingDate = date.today() #Use todays date as the Pricing Date
    PricingDate = date(2025,5,19) # For testing currently
    SDate = mfbd(PricingDate + timedelta(days=2),'US') #Calculating the Spot Start Date
    df = pd.DataFrame(data)
    df.columns = ['Year','Rate']
    i = int(i)
    if i>-1:
        df.iat[i,1]=df.iat[i,1]+0.0001*p
        
    df['MatDate'] = SDate 
    df['PmtDate'] = SDate

    #Populate Maturity Date and Payment Date adjusting for pay delay

    for index,row in df.iterrows():
        df.at[index,'MatDate'] = mfbd(row['MatDate'] + pd.offsets.DateOffset(years=row['Year']),'US')
        df.at[index,'PmtDate'] = mfbd(df.at[index,'MatDate'] + pd.offsets.DateOffset(days=2),'US') #SOFR Swaps have 2 days Payment Delay
    
    df['MatDate'] = pd.to_datetime(df['MatDate']).dt.date # To remove hh:mm:ss from Date
    df['PmtDate'] = pd.to_datetime(df['PmtDate']).dt.date
    df['DC'] = df['PmtDate'].diff(periods=1) #Compute Periodic day counts
    df.at[0,'DC']= pd.to_timedelta(df.at[0,'PmtDate']-SDate) #Compute first DC value from Start Date
    df['DC'] = pd.to_numeric(df['DC'].dt.days)
    df['DC']=df['DC']/360 #Since Daycount is Act/360 for USD SOFR
 
    import numpy as np
    df['DF'] = 1.000000
 
    SDate = mfbd(SDate,'US')

    #Just testing, delete later
    for index,row in df.iterrows():
        df.at[index,'DF'] = 1/(pow(1+row['Rate'],row['Year']))


    #Compute actual Curve DFs for USD SOFR
    #The for loop is massively simplified to solve a linear system of equations. Will write a full document on how this is calculated later. For queries in the interim email pushkargondane@gmail.com 
    dfdccumprodsum = 0
    for index,row in df.iterrows():
        df.at[index,'DF'] = (1 - df.at[index,'Rate']*dfdccumprodsum)/(1+df.at[index,'Rate']*df.at[index,'DC'])
        dfdccumprodsum = dfdccumprodsum + df.at[index,'DC']*df.at[index,'DF']

    #Compute Zeros for USD SOFR
    # Please note that zeros requires an assumption of the approach. Here we are going to try and match
    # it with that of bloomberg by assuming continuous compounding
    # Different systems implement this bit in a different manner


    df['Zero']=1.000000
    
    for index,row in df.iterrows():
        p = pd.to_timedelta(df.at[index,'PmtDate']-df.at[index,'MatDate'])
        t = float(p.days)/365 + df.at[index,'Year']
        #print(t)
        df.at[index,'Zero'] = np.log(1/df.at[index,'DF'])/t
        


    #print(df)
    return df
    



In [5]:
test=bootstrap(data)
print(test)

   Year      Rate     MatDate     PmtDate        DC        DF      Zero
0   1.0  0.040750  2026-05-21  2026-05-26  1.027778  0.959802  0.040474
1   2.0  0.037767  2027-05-21  2027-05-24  1.008333  0.927426  0.037517
2   3.0  0.037051  2028-05-22  2028-05-24  1.016667  0.895086  0.036878
3   4.0  0.037070  2029-05-21  2029-05-23  1.011111  0.862696  0.036873
4   5.0  0.037380  2030-05-21  2030-05-23  1.013889  0.830086  0.037204
5   6.0  0.037867  2031-05-21  2031-05-23  1.013889  0.797265  0.037727
6   7.0  0.038378  2032-05-21  2032-05-24  1.019444  0.764618  0.038295
7   8.0  0.038862  2033-05-23  2033-05-25  1.016667  0.732701  0.038851
8   9.0  0.039313  2034-05-22  2034-05-24  1.011111  0.701706  0.039336
9  10.0  0.039746  2035-05-21  2035-05-23  1.011111  0.671430  0.039813


Function D - Validate DF vs Alternate System like Bloomberg

![title][def]

[def]: SOFRBBG.png

In [ ]:
def compare(df):
    import pandas as pd
    bbgdata = import_excel('Market Data.xlsx','USDIRS','g3:h12')
    bbg = pd.DataFrame(bbgdata)
    bbg.columns=['BBGDF','BBGZero']
    comp = df[['Year','MatDate','PmtDate','DF','Zero']]
    comp['BBGDF'] = bbg['BBGDF'] #Need to find a better method to append columns as this is giving a warning 
    comp['BBGZero'] = bbg['BBGZero']
    comp['DF Gap(bps)']=(comp['BBGDF']-comp['DF'])*10000/comp['BBGDF']
    comp['Zero Gap(bps)']=(comp['BBGZero']-comp['Zero'])*10000/comp['BBGZero']
    print(comp)
    comp=comp.drop(columns=['Year','BBGDF','BBGZero','DF Gap(bps)','Zero Gap(bps)'])
    #Send the data back to excel 
    export_excel('Market Data.xlsx','USDIRS','L2:L11',comp)

In [19]:
compare(test)

   Year     MatDate     PmtDate        DF      Zero     BBGDF   BBGZero  \
0   1.0  2026-05-21  2026-05-26  0.959802  0.040474  0.960094  0.040502   
1   2.0  2027-05-21  2027-05-24  0.927426  0.037517  0.927482  0.037538   
2   3.0  2028-05-22  2028-05-24  0.895086  0.036878  0.895045  0.036826   
3   4.0  2029-05-21  2029-05-23  0.862696  0.036873  0.862658  0.036859   
4   5.0  2030-05-21  2030-05-23  0.830086  0.037204  0.830051  0.037193   
5   6.0  2031-05-21  2031-05-23  0.797265  0.037727  0.797234  0.037716   
6   7.0  2032-05-21  2032-05-24  0.764618  0.038295  0.764668  0.038271   
7   8.0  2033-05-23  2033-05-25  0.732701  0.038851  0.732677  0.038802   
8   9.0  2034-05-22  2034-05-24  0.701706  0.039336  0.701686  0.039303   
9  10.0  2035-05-21  2035-05-23  0.671430  0.039813  0.671413  0.039794   

   DF Gap(bps)  Zero Gap(bps)  
0     3.043136       6.936892  
1     0.600978       5.616469  
2    -0.453521     -14.162227  
3    -0.444027      -3.806096  
4    -0.424608

/var/folders/5g/4br4skc15dlb750j33xn06rh0000gn/T/ipykernel_1179/832407605.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp['BBGDF'] = bbg['BBGDF'] #Need to find a better method to append columns as this is giving a warning


Note that while the gap in DFs is miniscule (less than 1 bps vs BBG DFs) the zeros are slightly more different. They dont impact any calculations as the approach to calculate zeros simply depends on the base formula used for zeros. 

Valuation differences may arise if you choose to interpolate on zeros vs interpolate on DFs 

Function E - Pricing an Interest Rate Swap

In [ ]:
def priceIRS(i=-1,p=1):
    #Part A - Easy - Price the Fixed Leg of the IRS
    import pandas as pd
    from datetime import date,timedelta
    import holidays

    data = import_excel('Market Data.xlsx','USDIRS','A3:B12') #Import market data
    df = bootstrap(data,i,p) #Bootstrap the curve
    PricingDate = date(2025,5,19) # For testing currently
    SDate = mfbd(PricingDate + timedelta(days=2),'US') #Calculating the Spot Start Date
    #Add 1 as Discount Factor for Spot Date in the Discount Factors Data Frame
    df.loc[-1] = [0,0,SDate,SDate,0,1,0]
    df.index = df.index+1
    df = df.sort_index()

    fixed = import_excel('TradePrice.xlsx','SOFR','b3:g12')
    fixed = pd.DataFrame(fixed)
    fixed.columns=['SDate','EDate','PDate','Notional','Amort','FixRate']

    #Apply MFBD Calendar on the dates
    for index,row in fixed.iterrows():
        fixed.at[index,'SDate'] = mfbd(row['SDate'],'US')
        fixed.at[index,'EDate'] = mfbd(row['EDate'],'US')
        fixed.at[index,'PDate'] = mfbd(row['PDate'],'US') 

    fixed['DC']=fixed['EDate']-fixed['SDate']
    fixed['DC']=pd.to_timedelta(fixed['DC']).dt.days.astype(float)/360
    fixed['CF']=fixed['Notional']*fixed['FixRate']*fixed['DC']

    #Interpolate Discount Factors from Bootstrapping procedure


    df["PmtDate"] = df["PmtDate"].astype("datetime64[ns]")
    df['PmtDate']=df['PmtDate'].values.astype(float)
    df["MatDate"] = df["MatDate"].astype("datetime64[ns]")
    df['MatDate']=df['MatDate'].values.astype(float)

    import numpy as np
    fixed['DF']=np.interp(fixed['PDate'],df['PmtDate'],df['DF']) #Simple linear interpolation 
    fixed['NPV'] = fixed['DF']*fixed['CF']

    floating = import_excel('TradePrice.xlsx','SOFR','b16:i25')
    floating = pd.DataFrame(floating)
    floating.columns=['SDate','EDate','PDate','Notional','Amort','Index','Leverage','Spread']


    #Apply MFBD Calendar on the dates
    for index,row in floating.iterrows():
        floating.at[index,'SDate'] = mfbd(row['SDate'],'US')
        floating.at[index,'EDate'] = mfbd(row['EDate'],'US')
        floating.at[index,'PDate'] = mfbd(row['PDate'],'US')#Assumed standard Pay Delay of 2 Days

    floating['DC']=floating['EDate']-floating['SDate']
    floating['DC']=pd.to_timedelta(floating['DC']).dt.days.astype(float)/360
    floating['DF']=np.interp(floating['PDate'],df['PmtDate'],df['DF']) #Simple linear interpolation 
    floating['DFSdate']=np.interp(floating['SDate'],df['MatDate'],df['DF']) #Simple linear interpolation 
    floating['DFEdate']=np.interp(floating['EDate'],df['MatDate'],df['DF']) #Simple linear interpolation 
    floating['Forecast']=((floating['DFSdate']/floating['DFEdate'])-1)/floating['DC']

    #The first forecast rate is incorrect as it ignores the impact due to the 
    # daily fixings which have already occured. We need to make an adjustment for that
    fixings = import_excel('Market Data.xlsx','SOFRFix','C2:D348') #Import market data
    fixings = pd.DataFrame(fixings)
    fixings.columns=['Date','FixRate']

    filter=floating.iat[0,0] #Obtain the start date of the fixings
    fixings = fixings[fixings['Date']>=filter] #Note we have assumed that fixings data is latest available. 

    #We have also assumed that the fix for the 2 unknown days between last fixing and spot is also available
    #You can alternatively assume its the same as last fix and wont make much difference to overall calculations

    fixings['DC'] = fixings['Date'].diff(periods=1) #Compute Periodic day counts
    fixings.iat[0,2] = fixings.iat[0,0] - mfbd(fixings.iat[0,0]+timedelta(days=1),'US')
    fixings['DC']=fixings['DC']*-1
    fixings['Days']=fixings['DC']/timedelta(days=1)/360 #Using Act/360 Day count convention
    fixings['FixDays']=fixings['FixRate']*fixings['Days']/100

    floating.iat[0,12] = floating.iat[0,12] + fixings['FixDays'].sum()
    floating['NPV']=floating['Notional']*(floating['Leverage']*floating['Forecast']+floating['Spread'])*floating['DC']*floating['DF']


    return fixed['NPV'].sum()+floating['NPV'].sum()

Lets now generate the bucketed sensivitiy / DV01 for this trade

Approach #1 - Simpler which involves shifting the underlying curve up by 1 bps only

In [8]:
NetPV = priceIRS() #Compute the unperturbed PV first
print ("Current NPV : ", round(NetPV,2))
Dv01 = 0
for i in range(10):
    NewPV = priceIRS(i)
    print("Bucket ",i+1 ,":" , round(NewPV - NetPV,2))

    Dv01 = Dv01 + NewPV - NetPV
print ("Total Dv01 : ", round(Dv01,2))




Current NPV :  436822.06
Bucket  1 : 6.31
Bucket  2 : -8.53
Bucket  3 : -11.15
Bucket  4 : -16.54
Bucket  5 : -19.22
Bucket  6 : -23.43
Bucket  7 : -31.58
Bucket  8 : -33.03
Bucket  9 : -2769.42
Bucket  10 : -5318.7
Total Dv01 :  -8225.31


Approach #2 - Slightly more computationally intensive and involves shifting the underlying curve up and below by 1 bps and then taking their average

In [14]:
NetPV = priceIRS() #Compute the unperturbed PV first
print ("Current NPV : ", round(NetPV,2))
Dv01 = 0
for i in range(10):
    NewPVup = priceIRS(i,1) 
    NewPVdn = priceIRS(i,-1)
    print("Bucket ",i+1 ,":" , round((NewPVup - NewPVdn)/2,2))

    Dv01 = Dv01 + (NewPVup - NewPVdn)/2
print ("Total Dv01 : ", round(Dv01,2))

Current NPV :  436822.06
Bucket  1 : 6.31
Bucket  2 : -8.53
Bucket  3 : -11.15
Bucket  4 : -16.54
Bucket  5 : -19.23
Bucket  6 : -23.43
Bucket  7 : -31.56
Bucket  8 : -33.03
Bucket  9 : -2769.69
Bucket  10 : -5319.22
Total Dv01 :  -8226.09
